In [1]:
import numpy as np
import tflearn
import tensorflow as tf
import random
import json
import pickle
from utils import clean_pattern, define_network

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
with open('intents.json') as file:
    data = json.load(file)
print(data)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'hey'], 'responses': ['Hello! Good to see you!', 'Hi there, how can I help?', "Hey genius, what's up?"], 'context_set': ''}, {'tag': 'goodbye', 'patterns': ['cya', 'See you later', 'Goodbye', 'I am Leaving', 'Have a Good day', 'bye'], 'responses': ['See you soon pal...', 'Goodbye! Have a nice day.'], 'context_set': ''}, {'tag': 'booking', 'patterns': ['book room', 'do you have any rooms?', 'room reservation', 'I want to book room'], 'responses': ['Do you want a single room or multiple rooms?'], 'context_set': 'rooms_num'}, {'tag': 'one_room', 'patterns': ['single room', 'one room', '1 room'], 'responses': ['Okay, we got you covered for 1 room!'], 'context_filter': 'rooms_num'}, {'tag': 'multiple_rooms', 'patterns': ['many', 'multiple rooms', 'more than 1 rooms'], 'responses': ['Okay, please visit our website and book multiple rooms!'], 'context_filter': 'rooms_num'}, {'tag': 'name', 'pattern

In [3]:
# Some cleaning of data in intents.json
stemmed_words = []
tags = []
ignore_words = ['!', '?', '.']
corpus = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        stemmed_pattern = clean_pattern(pattern, ignore_words)
        stemmed_words.extend(stemmed_pattern)
        corpus.append((stemmed_pattern, intent['tag']))
    if intent['tag'] not in tags:
        tags.append(intent['tag'])

# remove duplicates and sort
stemmed_words = sorted(list(set(stemmed_words)))
tags = sorted(list(set(tags)))

print(stemmed_words)
print(tags)
print(corpus)

["'s", ',', '1', 'a', 'address', 'am', 'any', 'anyon', 'ar', 'awesom', 'book', 'bye', 'cal', 'cya', 'day', 'do', 'doe', 'find', 'for', 'giv', 'good', 'goodby', 'hav', 'hello', 'help', 'hey', 'hi', 'hotel', 'how', 'i', 'is', 'lat', 'leav', 'loc', 'many', 'me', 'mor', 'multipl', 'nam', 'of', 'on', 'plac', 'reserv', 'room', 'see', 'should', 'singl', 'than', 'thank', 'that', 'the', 'ther', 'thi', 'to', 'want', 'what', 'wher', 'you']
['address', 'booking', 'goodbye', 'greeting', 'multiple_rooms', 'name', 'one_room', 'thanks']
[(['hi'], 'greeting'), (['how', 'ar', 'you'], 'greeting'), (['is', 'anyon', 'ther'], 'greeting'), (['hello'], 'greeting'), (['hey'], 'greeting'), (['cya'], 'goodbye'), (['see', 'you', 'lat'], 'goodbye'), (['goodby'], 'goodbye'), (['i', 'am', 'leav'], 'goodbye'), (['hav', 'a', 'good', 'day'], 'goodbye'), (['bye'], 'goodbye'), (['book', 'room'], 'booking'), (['do', 'you', 'hav', 'any', 'room'], 'booking'), (['room', 'reserv'], 'booking'), (['i', 'want', 'to', 'book', 'ro

In [4]:
# Creating numeric features and labels out of cleaned data
X = []
y = []
for item in corpus:
    bag = [] #array of 1 and 0. 1 if stemmed word is present in stemmed pattern
    stemmed_pattern = item[0]
    for w in stemmed_words:
        if w in stemmed_pattern:
            bag.append(1)
        else:
            bag.append(0)

    tags_row = [] #array of 1 and 0. 1 for current tag and for everything else 0.
    current_tag = item[1]
    for tag in tags:
        if tag == current_tag:
            tags_row.append(1)
        else:
            tags_row.append(0)

    #for each item in corpus, X will be array indicating stemmed words and y array indicating tags
    X.append(bag)
    y.append(tags_row) 

X = np.array(X)
y = np.array(y)
print(X)
print(y)

# saving variables in pickle to be used by main.py
with open('saved_variables.pickle', 'wb') as file:
    pickle.dump((stemmed_words, tags, ignore_words, X, y), file)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]]
[[0 0 0 1 0 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 1 0 0 0 0 0]
 [0 0 1 0 0 0 0 0]
 [0 0 1 0 0 0 0 0]
 [0 0 1 0 0 0 0 0]
 [0 0 1 0 0 0 0 0]
 [0 0 1 0 0 0 0 0]
 [0 1 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 1 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0]]


In [5]:
model = define_network(X, y)
model.fit(X, y, n_epoch=1120, batch_size=8, show_metric=True) 
model.save("chatbot_model.tflearn")

Training Step: 5599  | total loss: 0.02223 | time: 0.008s
| Adam | epoch: 1120 | loss: 0.02223 - acc: 0.9994 -- iter: 32/34
Training Step: 5600  | total loss: 0.02170 | time: 0.010s
| Adam | epoch: 1120 | loss: 0.02170 - acc: 0.9995 -- iter: 34/34
--
INFO:tensorflow:F:\ChatBot\chatbot_model.tflearn is not in all_model_checkpoint_paths. Manually adding it.
